In [1]:
import torch
import numpy as np
from torch import nn

# RNN

[https://pytorch.org/docs/stable/generated/torch.nn.RNN.html](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# 배치 크기: 3, 시퀀스 길이: 5, 입력 크기: 256의 더미 입력 생성
input_tensor = torch.randn(3, 5, 256)

## d는 bidirectional=True일 때 2
## batch_first = False -> [seq_len, batch_size, d * output_dim]
## batch_first = True -> [batch_size, seq_len, d * output_dim]
rnn = nn.RNN(input_size=256,
             hidden_size=256,
             nonlinearity="tanh",
             batch_first=True)

output, h_n = rnn(input_tensor) ## output은 각 t마다 얻은 output feature에 해당.

## output 텐서는 RNN의 마지막 레이어에서 각 타임스텝(t)마다 생성된 출력 특징들을 포함.
## 길이가 5인 시퀀스 데이터를 rnn에 입력했을 때 0~4까지의 hidden_state가 발생하니까 output의 두번째 차원이 5
print(output.shape) ## [3, 5, 256], [batch, seq_len, hidden_dim]

## h_n 텐서는 RNN의 마지막 타임스텝에서 각 배치 항목에 대한 최종 은닉 상태를 포함.
## 배치별로 마지막 t번째 hidden state를 가져오니까 [1, 3, 256]
print(h_n.shape) ## 

torch.Size([3, 5, 256])
torch.Size([1, 3, 256])


In [4]:
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1, nonlinearity='tanh', batch_first=True):
        super().__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.nonlinearity = nonlinearity
        self.output_dim = output_dim

        self.rnn = nn.RNN(input_size=self.input_dim,
                          hidden_size=self.hidden_dim,
                          nonlinearity=self.nonlinearity,
                          batch_first=batch_first)
        
        self.output_layer = nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self, x, device):
        h0 = torch.zeros(1, x.size(0), self.hidden_dim).to(device) ## hidden_state 초기화.

        out, h_n = self.rnn(x, h0)

        out = self.output_layer(out[:, -1, :]) ## 마지막 시간 단계의 hidden_state만 가져와 선형변환.

        return out

In [5]:
model = RNN(input_dim=2, hidden_dim=20, output_dim=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [6]:
inputs = torch.from_numpy(np.array([[[1, 2], [3, 4], [5, 6]]], dtype=np.float32)).to(device)

for epoch in range(300): # 300번의 에폭 동안 학습을 진행
    model.zero_grad() # 기울기를 0으로 초기화
    outputs = model(inputs, device) # 모델에 입력을 전달하고 출력을 받음

    loss = criterion(outputs, torch.tensor([1]).to(device))  # 더미 타겟 데이터로 손실(loss)을 계산
    loss.backward() # 역전파를 통해 기울기를 계산

    optimizer.step() # 최적화 알고리즘을 통해 파라미터를 업데이트

    if (epoch+1) % 30 == 0: # 30 에폭마다 손실을 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 300, loss.item()))

Epoch [30/300], Loss: 0.0973
Epoch [60/300], Loss: 0.0303
Epoch [90/300], Loss: 0.0155
Epoch [120/300], Loss: 0.0100
Epoch [150/300], Loss: 0.0072
Epoch [180/300], Loss: 0.0056
Epoch [210/300], Loss: 0.0045
Epoch [240/300], Loss: 0.0037
Epoch [270/300], Loss: 0.0031
Epoch [300/300], Loss: 0.0027


# LSTM

[https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM)

- LSTM(Long Short-Term Memory)에서의 cell state는 hidden state를 보완하고, 더 나은 기억과 정보를 유지하기 위한 역할.
- 일반적인 RNN은 시퀀스가 길어질수록 이전 정보가 제대로 유지되지 않는 문제가 있다.
- forgot gate, input gate, output gate를 통해 어떤 정보를 기억하고 어떤 정보를 잊을지 결정.

In [8]:
# 배치 크기: 3, 시퀀스 길이: 5, 입력 크기: 256의 더미 입력 생성
input_tensor = torch.randn(3, 5, 256)

## d는 bidirectional=True일 때 2
## batch_first = True -> [batch_size, seq_len, d * output_dim]
lstm = nn.LSTM(input_size=256,
               hidden_size=256,
               batch_first=True)

output, (h_n, c_n) = lstm(input_tensor)

## output 텐서는 RNN의 마지막 레이어에서 각 타임스텝(t)마다 생성된 출력 특징들을 포함.
## 길이가 5인 시퀀스 데이터를 rnn에 입력했을 때 0~4까지의 hidden_state가 발생하니까 output의 두번째 차원이 5
print(output.shape) ## [3, 5, 256], [batch, seq_len, hidden_dim]

## h_n 텐서는 RNN의 마지막 타임스텝에서 각 배치 항목에 대한 최종 은닉 상태를 포함.
## 배치별로 마지막 t번째 hidden state를 가져오니까 [1, 3, 256]
print(h_n.shape)

## c_n 텐서는 시퀀스의 마지막 타임스텝에서 각 LSTM 레이어의 최종 cell state를 포함.
## 배치별로 마지막 t번째 cell state를 가져오니까 [1, 3, 256]
print(c_n.shape)

torch.Size([3, 5, 256])
torch.Size([1, 3, 256])
torch.Size([1, 3, 256])


In [12]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1, nonlinearity='tanh', batch_first=True):
        super().__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.nonlinearity = nonlinearity
        self.output_dim = output_dim

        self.rnn = nn.LSTM(input_size=self.input_dim,
                          hidden_size=self.hidden_dim,
                          batch_first=batch_first)
        
        self.output_layer = nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self, x, device):
        h0 = torch.zeros(1, x.size(0), self.hidden_dim).to(device) ## hidden_state 초기화.
        c0 = torch.zeros(1, x.size(0), self.hidden_dim).to(device) ## cell_state 초기화.

        out, (h_n, cn) = self.rnn(x, (h0, c0))

        out = self.output_layer(out[:, -1, :]) ## 마지막 timestemp의 hidden_state만 가져와 선형변환.

        return out

In [13]:
model = LSTM(input_dim=2, hidden_dim=20, output_dim=2).to(device) # LSTM 모델을 생성
criterion = nn.CrossEntropyLoss() # 손실 함수로 CrossEntropyLoss를 사용
optimizer = torch.optim.Adam(model.parameters()) # 최적화 알고리즘으로 Adam을 사용

for epoch in range(300): # 300회의 에포크동안 학습을 진행
    model.zero_grad() # 기울기를 0으로 초기화
    outputs = model(inputs, device) # 모델에 입력을 전달하고 출력을 받음
    loss = criterion(outputs, torch.tensor([1]).to(device))  # 예시로 사용할 목표 텐서 생성
    loss.backward() # 역전파를 수행하여 기울기를 계산
    optimizer.step() # 최적화 알고리즘을 통해 파라미터 업데이트

    if (epoch+1) % 30 == 0: # 30 에포크마다 손실을 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 300, loss.item()))

Epoch [30/300], Loss: 0.4644
Epoch [60/300], Loss: 0.1944
Epoch [90/300], Loss: 0.0654
Epoch [120/300], Loss: 0.0289
Epoch [150/300], Loss: 0.0165
Epoch [180/300], Loss: 0.0108
Epoch [210/300], Loss: 0.0078
Epoch [240/300], Loss: 0.0059
Epoch [270/300], Loss: 0.0047
Epoch [300/300], Loss: 0.0039


# GRU

[https://pytorch.org/docs/stable/generated/torch.nn.GRU.html#torch.nn.GRU](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html#torch.nn.GRU)

In [14]:
# 배치 크기: 3, 시퀀스 길이: 5, 입력 크기: 256의 더미 입력 생성
input_tensor = torch.randn(3, 5, 256)

## d는 bidirectional=True일 때 2
## batch_first = True -> [batch_size, seq_len, d * output_dim]
rnn = nn.GRU(input_size=256,
             hidden_size=256,
             batch_first=True)

output, h_n = rnn(input_tensor)

## output 텐서는 RNN의 마지막 레이어에서 각 타임스텝(t)마다 생성된 출력 특징들을 포함.
## 길이가 5인 시퀀스 데이터를 rnn에 입력했을 때 0~4까지의 hidden_state가 발생하니까 output의 두번째 차원이 5
print(output.shape) ## [3, 5, 256], [batch, seq_len, hidden_dim]

## h_n 텐서는 RNN의 마지막 타임스텝에서 각 배치 항목에 대한 최종 은닉 상태를 포함.
## 배치별로 마지막 t번째 hidden state를 가져오니까 [1, 3, 256]
print(h_n.shape) ## 

torch.Size([3, 5, 256])
torch.Size([1, 3, 256])


In [16]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1, nonlinearity='tanh', batch_first=True):
        super().__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.nonlinearity = nonlinearity
        self.output_dim = output_dim

        self.rnn = nn.GRU(input_size=self.input_dim,
                          hidden_size=self.hidden_dim,
                          batch_first=batch_first)
        
        self.output_layer = nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self, x, device):
        h0 = torch.zeros(1, x.size(0), self.hidden_dim).to(device) ## hidden_state 초기화.

        out, h_n = self.rnn(x, h0)

        out = self.output_layer(out[:, -1, :]) ## 마지막 timestemp의 hidden_state만 가져와 선형변환.

        return out

In [18]:
# GRU 학습
model = GRU(input_dim=2, hidden_dim=20, output_dim=2).to(device) # GRU 모델 인스턴스 생성
criterion = nn.CrossEntropyLoss() # 손실 함수로 CrossEntropyLoss를 사용
optimizer = torch.optim.Adam(model.parameters()) # 최적화 알고리즘으로 Adam을 사용

for epoch in range(300):
    model.zero_grad()
    outputs = model(inputs, device)
    loss = criterion(outputs, torch.tensor([1]).to(device))  # A dummy target example
    loss.backward()
    optimizer.step()

    if (epoch+1) % 30 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 300, loss.item()))

Epoch [30/300], Loss: 0.2199
Epoch [60/300], Loss: 0.0783
Epoch [90/300], Loss: 0.0301
Epoch [120/300], Loss: 0.0150
Epoch [150/300], Loss: 0.0097
Epoch [180/300], Loss: 0.0071
Epoch [210/300], Loss: 0.0056
Epoch [240/300], Loss: 0.0045
Epoch [270/300], Loss: 0.0038
Epoch [300/300], Loss: 0.0032
